### Project Title: Financial Data Cleaning Pipeline

## Overview:
The Financial Data Cleaning Pipeline is a foundational component of any quantitative finance or algorithmic trading system. Raw market data—whether from equities, forex, crypto, or derivatives—is often incomplete, noisy, and unnormalized. This project focuses on developing a robust, scalable pipeline using Python, pandas, and NumPy to prepare financial datasets for predictive modeling, algorithmic strategies, and risk analytics. The pipeline addresses missing data, outliers, scaling, and ensures structural integrity for downstream tasks.

## Objectives:

Acquire and explore real-world market datasets

Identify and remove outliers and anomalies

Handle missing or corrupt data systematically

Normalize or scale numerical data for ML compatibility

Output a clean, validated dataset for modeling or backtesting

## Technologies Used:

Python 3.x

pandas (data wrangling)

NumPy (numeric operations)

yfinance (market data API)

scikit-learn (normalization)

matplotlib/seaborn (visual diagnostics)

### STEP 1. Dataset Acquisition:

We use yfinance to download historical financial data directly from Yahoo Finance. Example: SPY ETF as a proxy for S&P 500 index.

In [203]:
import yfinance as yf

# Download data for SPY (2015–2025)
ticker=['AAPL']
df = yf.download(ticker, start="2000-01-01", end="2025-07-01")
df.to_csv("spy_raw.csv")
df.head()

C:\Users\gbanwat\AppData\Local\Temp\ipykernel_2328\1563363508.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2000-01-01", end="2025-07-01")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2000-01-03,0.841048,0.845274,0.764034,0.787983,535796800
2000-01-04,0.770139,0.831186,0.760277,0.813341,512377600
2000-01-05,0.781408,0.830716,0.773895,0.779530,778321600
2000-01-06,0.713787,0.803949,0.713787,0.797375,767972800
2000-01-07,0.747598,0.758868,0.717544,0.725057,460734400


### STEP 2. Data Loading & Initial Exploration:

Identify missing values and range anomalies

Visualize using df.plot() or seaborn

In [204]:
import numpy as np
import pandas as pd
df=pd.read_csv('spy_raw.csv')
df.set_index('Price',inplace=True)
df.index.name='Date'
desired_columns=['Close','High','Low','Open','Volume']
df=df.iloc[2:,:]
df.columns=pd.MultiIndex.from_product([desired_columns,ticker],names=['Metric','Ticker'])
df.columns=df.columns.swaplevel(0,1)
df.sort_index(axis=1,level=0,inplace=True)
#df.head()
df.dtypes

Ticker  Metric
AAPL    Close     object
        High      object
        Low       object
        Open      object
        Volume    object
dtype: object

In [205]:
# Changing All the columns to required dtypes
price_cols=['Close','High','Low','Open']
vol_cols=['Volume']
for (ticker,col) in df.columns:
    if col in price_cols:
        df[(ticker,col)]=df[(ticker,col)].astype(float).round(4)
    elif col in vol_cols:
        df[(ticker,col)]=pd.to_numeric(df[(ticker,col)],errors='coerce')
df.dtypes

Ticker  Metric
AAPL    Close     float64
        High      float64
        Low       float64
        Open      float64
        Volume      int64
dtype: object

In [206]:
df.head()

Ticker        AAPL                                   
Metric       Close    High     Low    Open     Volume
Date                                                 
2000-01-03  0.8410  0.8453  0.7640  0.7880  535796800
2000-01-04  0.7701  0.8312  0.7603  0.8133  512377600
2000-01-05  0.7814  0.8307  0.7739  0.7795  778321600
2000-01-06  0.7138  0.8039  0.7138  0.7974  767972800
2000-01-07  0.7476  0.7589  0.7175  0.7251  460734400

### STEP 3. Handling Missing Values:

In [207]:
# Check nulls
df.isnull().sum()

# Forward fill followed by backward fill (for gaps in time series)
df = df.ffill().bfill()

print(df[('AAPL','Close')].dtype)

float64


### STEP 4: Outlier Detection & Removal-
#### Option 1 : Interquartile Range (IQR) Method
#### Option 2 : Z-Score Method

Visual inspection using boxplots: sns.boxplot(df['Close'])

In [208]:
## Option 1:
#for ticker,cols in df.columns:
#    if cols=='Close':
#        Q1 = df[(ticker,'Close')].quantile(0.25)
#        Q3 = df[(ticker,'Close')].quantile(0.75)
#        IQR = Q3 - Q1
#        lower = Q1 - 1.5 * IQR
#        upper = Q3 + 1.5 * IQR

In [175]:
Q1 = df[('AAPL','Close')].quantile(0.25)
Q3 = df[('AAPL','Close')].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
print(Q1,Q3,IQR,lower,upper)

# Filter out outliers in 'Close'
df = df[(df[('AAPL','Close')] >= lower) & (df[('AAPL','Close')] <= upper)]

2.0539 47.66335 45.60945 -66.360275 116.07752500000001


In [209]:
## Option 2:
#(6365, 4)
#(6411, 4)
from scipy.stats import zscore

z = np.abs(zscore(df[[('AAPL','Open'),('AAPL','High'), ('AAPL','Low'), ('AAPL','Close')]]))
df = df[(z < 3).all(axis=1)]
z.shape

(6411, 4)

### STEP 5: Normalization / Feature Scaling
Normalization ensures values fall in a similar range, which is crucial for algorithms sensitive to scale.

Alternative: StandardScaler for z-score normalization

In [177]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols_to_scale = [('AAPL','Open'),('AAPL', 'High'), ('AAPL','Low'), ('AAPL','Close'), ('AAPL','Volume')]
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
df.head()

Ticker          AAPL                                        
Metric         Close      High       Low      Open    Volume
Date                                                        
2000-01-03  0.005564  0.005447  0.004960  0.005026  0.067250
2000-01-04  0.004951  0.005328  0.004928  0.005240  0.064077
2000-01-05  0.005049  0.005324  0.005046  0.004954  0.100102
2000-01-06  0.004464  0.005098  0.004525  0.005106  0.098700
2000-01-07  0.004756  0.004720  0.004557  0.004493  0.057082

### STEP 6: Data Validation & Export

Ensure no missing or extreme values

Visualize post-cleaning trends with line plots

In [ ]:
# Final checks
print(df.isnull().sum())
print(df.describe())

# Save cleaned data
df.to_csv("spy_cleaned.csv")